## Google Photos API

## Create virtualenv and install required packages

1. Open terminal and navigate to your working directory

2. Create a virtual environment `python3 -m venv venv`

3. Add your virtual environment to Jupyter: `python3 -m ipykernel install --user --name=venv`

4. Start jupyter notebook or jupyter lab: `jupyter lab .`

## Enable Google API 



5. Enable Google Photos API Service

   1. Go to the Google API Console [https://console.cloud.google.com/](https://console.cloud.google.com/). 
   2. From the menu bar, select a project or create a new project.
      ![](read_me_img/create_a_project_google.png)
   3. To open the Google API Library, from the Navigation menu, select APIs & Services > Library. 
   4. Search for "Google Photos Library API". Select the correct result and click "enable". If its already enabled, click "manage"
   5. Afterwards it will forward you to the "Photos API/Service details" page (https://console.cloud.google.com/apis/credentials)


6. Create API/OAuth credentials

   1. On the left side at the Google Photos API Service page click on Credentials
   2. Click on "Create Credentials" and create a OAuth client ID
   3. As application type I am choosing "Desktop app" and give your client you want to use to call the API a name
   4. Download the JSON file to the created credentials, rename it to "client_secret.json" and save it in the folder "credentials"


7. Configure "OAuth consent screen" ([Source](https://stackoverflow.com/questions/65184355/error-403-access-denied-from-google-authentication-web-api-despite-google-acc))

   1. Go back to the Photos API Service details page and click on "[OAuth consent screen](https://console.cloud.google.com/apis/credentials/consent)" on the left side (below "Credentials") 
   2. Add a Test user: Use the email of the account you want to use for testing the API call
   

## Install required packages

In [58]:
%%capture capt 
#saves the output to variable capt, to print output capt.stdout, capt.stderr
!pip install google_auth_oauthlib
!pip install pip install google-api-python-client

In [59]:
import pandas as pd

# import customized modules
import sys
import os

import sys
sys.path.append(r'./helper_scripts')
from google_photos_api import GooglePhotosApi

## Utilize Google Photos Library API

5. Use the customized funtions from "google_photos_api.py" to create a service for the first time:

    1. Initialize GooglePhotosApi `google_photos_api = GooglePhotosApi()`

    2. Create Service using the `client_secret.json` file: `service = google_photos_api.create_service()`
        
        
       <b>Calling the API for the first time:</b>
       1. Google will ask you if you want to grant the App the required permissions you defined with the scope:
       ![](read_me_img/sign_in_google_acc.png)
       2. Since its just a test app at the moment, Google will make you aware of that > Click on "Continue"
       3. Once you granted the app the required permissions, you will see a "token_......pickle" file created in the folder "credentials". This token file will be used for future calls.

In [60]:
import pickle
import os
from google_auth_oauthlib.flow import Flow, InstalledAppFlow
from googleapiclient.discovery import build
#from googleapiclient.http import MediaFileUpload
from google.auth.transport.requests import Request
import requests

class GooglePhotosApi:
    def __init__(self,
                 api_name = 'photoslibrary',
                 client_secret_file= r'./credentials/client_secret.json',
                 api_version = 'v1',
                 scopes = ['https://www.googleapis.com/auth/photoslibrary']):
        '''
        Args:
            client_secret_file: string, location where the requested credentials are saved
            api_version: string, the version of the service
            api_name: string, name of the api e.g."docs","photoslibrary",...
            api_version: version of the api

        Return:
            service:
        '''

        self.api_name = api_name
        self.client_secret_file = client_secret_file
        self.api_version = api_version
        self.scopes = scopes
        self.cred_pickle_file = f'./credentials/token_{self.api_name}_{self.api_version}.pickle'

        self.cred = None

    def run_local_server(self):
        # is checking if there is already a pickle file with relevant credentials
        if os.path.exists(self.cred_pickle_file):
            with open(self.cred_pickle_file, 'rb') as token:
                self.cred = pickle.load(token)

        # if there is no pickle file with stored credentials, create one using google_auth_oauthlib.flow
        if not self.cred or not self.cred.valid:
            if self.cred and self.cred.expired and self.cred.refresh_token:
                self.cred.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(self.client_secret_file, self.scopes)
                self.cred = flow.run_local_server()

            with open(self.cred_pickle_file, 'wb') as token:
                pickle.dump(self.cred, token)
        
        return self.cred


In [61]:
# initialize photos api and create service
google_photos_api = GooglePhotosApi()
creds = google_photos_api.run_local_server()

### Get list of all items on a specfic day

Use requests python module to send http requests to the Media Items API

Sources:
* https://stackoverflow.com/questions/56294506/mediaitems-search-next-returns-400
* https://developers.google.com/photos/library/reference/rest/v1/mediaItems/search?apix_params=%7B%22resource%22%3A%7B%22filters%22%3A%7B%22dateFilter%22%3A%7B%22dates%22%3A%5B%7B%22day%22%3A1%2C%22month%22%3A1%2C%22year%22%3A2022%7D%5D%7D%7D%7D%7D

Send a post request to the media API to get a list with all items. Since the API return is limited to 100 items, the search is narrowed down to one day. With this, the call would only have a problem if more than 100 images were made/uploaded in one day.

In [62]:
import json
import requests

def get_response_from_medium_api(year, month, day):
    url = 'https://photoslibrary.googleapis.com/v1/mediaItems:search'
    payload = {
                  "filters": {
                    "dateFilter": {
                      "dates": [
                        {
                          "day": day,
                          "month": month,
                          "year": year
                        }
                      ]
                    }
                  }
                }
    headers = {
        'content-type': 'application/json',
        'Authorization': 'Bearer {}'.format(creds.token)
    }
    
    res = requests.request("POST", url, data=json.dumps(payload), headers=headers)
    
    return(res)

Now use the response of the API to write the results and required metadata into a data frame.

In [89]:
def list_of_media_items(year, month, day, media_items_df):
    '''
    Args:
        year, month, day: day for the filter of the API call 
        media_items_df: existing data frame with all find media items so far
    Return:
        media_items_df: media items data frame extended by the articles found for the specified tag
    '''

    # create request for specified date
    response = get_response_from_medium_api(year, month, day)

    try:
        for item in response.json()['mediaItems']:
            item_df = pd.DataFrame(item)
            item_df = item_df.rename(columns={"mediaMetadata": "creationTime"})
            item_df.set_index('creationTime')
            item_df = item_df[item_df.index == 'creationTime']
            #print(item_df)
            #append the existing media_items data frame
            media_items_df = pd.concat([media_items_df, item_df])
    except:
        print(response.text)

    return(media_items_df)

In [ ]:
str(sdate)

Define a list of all dates from start date to today and execute the API call for all dates that have not already been queried.

In [6]:
import pandas
from datetime import date, timedelta

#create a list with all dates between start date and today
sdate = date(2020,4,16)   # start date
edate = date.today()
date_list = pandas.date_range(sdate,edate-timedelta(days=1),freq='d')
print(date_list)

media_items_df = pd.DataFrame()

for date in date_list:
    
    
    media_items_df = list_of_media_items(year = date.year, month = date.month, day = date.day, media_items_df = media_items_df)

#save a list of all media items to a csv file
media_items_df.to_csv(f'./media_item_list/item-list-{str(sdate)}.csv', index=True)
#media_items_df.to_csv('item-list-' + str(sdate) + r'.csv', index=True)
media_items_df

DatetimeIndex(['2020-04-16', '2020-04-17', '2020-04-18', '2020-04-19',
               '2020-04-20', '2020-04-21', '2020-04-22', '2020-04-23',
               '2020-04-24', '2020-04-25',
               ...
               '2022-04-15', '2022-04-16', '2022-04-17', '2022-04-18',
               '2022-04-19', '2022-04-20', '2022-04-21', '2022-04-22',
               '2022-04-23', '2022-04-24'],
              dtype='datetime64[ns]', length=739, freq='D')


NameError: name 'pd' is not defined

## Download images

In [ ]:
import requests
url = media_items['mediaItems'][0]['productUrl']
respons = requests.get(url)
file_name = media_items['mediaItems'][0]['filename']
with open(os.path.join(destination_folder, file_name), 'wb') as f:
    f.write(response.content)
    f.close()

## Upload files to AWS

In [ ]:
import boto3

client = boto3.client('s3', region_name='us-west-2')

client.upload_file('images/image_0.jpg', 'mybucket', 'image_0.jpg')

In [1]:
1+4

5